In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import torch
import numpy as np

from pathlib import Path
from transformers import MimiModel, AutoFeatureExtractor

In [ ]:
#

from overfit_trial.data_models import (
    MimiChannelArchive,
)

In [ ]:
channel_id_pair = ["V00_S0696_I00000375_P0844A", "V00_S0696_I00000375_P0847"]
num_quantizers = 32
SAMPLE_RATE_HZ = 24000

In [ ]:
%%capture

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MimiModel.from_pretrained("kyutai/mimi", num_quantizers=num_quantizers)
feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")

if device.type == "cuda":
    model.to(device)

In [ ]:
repo_root = Path(os.getcwd()).parent

channel1 = MimiChannelArchive(
    channel_id=channel_id_pair[0],
    npz_path=Path(repo_root / "asset" / "single_pair_dataset" / f"{channel_id_pair[0]}.{num_quantizers}q.npz"),
)
channel2 = MimiChannelArchive(
    channel_id=channel_id_pair[1],
    npz_path=Path(repo_root / "asset" / "single_pair_dataset" / f"{channel_id_pair[1]}.{num_quantizers}q.npz"),
)

In [ ]:
user = channel1.load_codes()

In [ ]:
user_codes = user.to_torch()

In [ ]:
model_output = model.decode(user_codes.unsqueeze(0).to(device))
audio_arr = model_output.audio_values.detach().cpu().squeeze().numpy()
# Audio(audio_arr, rate=SAMPLE_RATE_HZ)

In [ ]:
user_codes.shape

In [ ]:
sil_codes = model.encode(torch.from_numpy(np.zeros_like(audio_arr)).unsqueeze(0).unsqueeze(0).to(device))

In [ ]:
sil_codes.audio_codes.shape

In [ ]:
sil_codes.audio_codes[:, :, 0:10]